# 👊 CHAPTER 12 다중 머신과 장치를 위한 분산 텐서플로우  


#### 🙋 Chapter Manager : 박선재  🙎 Contents Add :   설민욱                       

다중 장치 - 다중 머신 - 큐 - 신경망 병렬화

In [2]:
import tensorflow  as tf
tfconf=tf.ConfigProto()
tfconf.gpu_options.per_process_gpu_memory_fraction=0.5
tfconf.log_device_placement=True

지금까지 가중치 초기화, 배치 정규화, 정교한 옵티마이저, 다양한 모델들로 훈련 속도와 예측 속도를 높일 수 있을 기술들을 봤다. 그러나 하나의 cpu만으로는 대규모 신경망을 훈련시키기는 힘들다. 이를 위해 훈련을 병렬화 할 수 있다.

## 👊 12.1 단일 머신의 다중 장치
일반적으로 로컬에서 훈련을 시키면 cpu를 활용하여 훈련한다. 그러나 훈련에 사용되는 데이터에 실수가 많으므로 gpu를 활용하여 속도를 올릴 수 있다.  
tensorflow에서 GPU를 사용하기 위해서는 CUDA와 cuDNN이 필요하다.  
device:GPU:N 을 통하여 사용자의 프로세서에 손쉽게 할당하고 제어할 수 있다.  


In [1]:
from IPython.display import Image
Image(url="img/cuda.jpg",height=300)

In [3]:
tf.reset_default_graph()
with tf.device("/device:GPU:0"):
  a=tf.constant(1.0,name="a")
with tf.Session(config=tfconf):
  print(a.eval())

Device mapping:

1.0


### 👊 12.1.2 GPU RAM 관리
tensorflow는 계산 그래프가 실행될 때 모든 gpu의 ram을 확보하므로 한 머신에 여러 그래프를 올릴 경우 오류가 발생한다.  
이를 위해 한 그래프에 gpu를 제한하여 할당하거나 gpu의 메모리를 제한하여 사용할 수 있다.  


In [ ]:
# 그래프에 gpu 할당
# $ CUDA_VISIBLE_DEVICES=0,1 python3 program_1.py
# $ CUDA_VISIBLE_DEVICES=2,3 python3 program_2.py

#메모리 제한
config=tf.ConfigProto()
config.gpu_option.per_process_gpu_memory_fraction = 0.4
session=tf.Session(config=config)
session.close()

### 👊 12.1.3 장치에 연산 배치하기
단순 배치는 device 함수를 통하여 할당할 수 있다. 이 device 함수의 인자로는 장치명이나 장치명을 반환하는 함수가 들어가 동적으로 배치할 수 있다.  
텐서 플로우는 정수 변수에 대한 gpu커널을 가지고 있지 않아 정수 변수를 gpu에 할당하려 하면 오류가 난다. allow_soft_placement 환경설정을 True로 하면 gpu에 할당할 시 자동으로 cpu에 할당한다.  
tensorflow 2.0 에서는 distribute 모듈에 동적 배치자가 있어 뒤에서 소개할 분산 처리 기법들 구현돼있다.

In [ ]:
config.allow_soft_placemnt=True

### 👊 12.1.4 병렬 실행  
텐서플로우는 연산의 의존성 카운터를 계산하여 0이 되면 평가큐에 추가하여 평가를 한다. 즉 그래프에 리프 노드에 해당되는 연산부터 실행되고 이 연산이 실행 되어 위 쪽의 의존성 카운를 감소시켜 평가큐에 추가되고 이 위의 노드가 실행된다.
이 연산이 cpu에서 일어날 경우 inter-op 스레드 풀로 들어가 여러 연산이 동시에 처리 될 것이고 이 연산 중 멀티 스레드 커널이 있다면 intra-op 스레드 풀에서 더 빠르게 연산 된다.  
gpu의 경우 대부분의 연산에 gpu커널이 존재하여 연산이 순서대로 평가된다.  

### 👊 12.1.5 제어 의존성
나중에 사용될 연산을 미리하는 것은 불필요한 메모리 소비가 일어날 것이다. contorl_dependencies 함수를 이용하여 의존성을 추가함으로 불필요한 자원 소모를 막을 수 있다.

In [4]:
tf.reset_default_graph()
a=tf.constant(1.0)
b=a+1.0
with tf.control_dependencies([a,b]):
  x=tf.constant(2.0)
z=b+x
with tf.Session():
  print(z.eval())

4.0


## 👊 12.2 다중 머신의 다중 장치  
지금까지는 단일 머신의 장치들을 활용하여 병렬 처리를 하였다. 그러나 이를 확장시켜 서버에서 여러 머신을 활용한다면 대규모 훈련을 더 효율적으로 할 수 있다.  
하나의 그래프는 클러스터 위에서 실행된다.  
클러스터는 ps(parameter server)와 worker라는 job으로 구성되어있다.  
job은 태스크로 이루어져있다. 이 태스크는 하나의 서버를 말한다.  
다음 코드처럼 하나의 머신에서 여러 서버를 열 수도 있지만 실제 서비스에서는 권장되지 않는다.  
서버를 열경우 이 서버는 마스터 서비스와 워커 서비스를 제공한다. 마스터 서비스는 클라이어트가 세션을 열고 그래프를 실행시킬 수 있게 도와주고 워커 서비스는 실제로 로컬 장소에서 계산을 실행하고 결과를 받는다.  
이는 클라이언트에서 어떤 서버에서도 세션을 열 수있게 한다.  

In [6]:
clusterspec=tf.train.ClusterSpec({"ps":["localhost:2221"],"worker":["localhost:2222","localhost:2223"]})
server1=tf.train.Server(clusterspec,job_name="ps",task_index=0)
server2=tf.train.Server(clusterspec,job_name="worker",task_index=0)
server3=tf.train.Server(clusterspec,job_name="worker",task_index=1)

In [7]:
tf.reset_default_graph()
with tf.device("/job:ps"):
  a=tf.Variable(2.0,name="a")
  init=tf.global_variables_initializer()
add=tf.assign_add(a,2.0)
with tf.Session("grpc://localhost:2221",config=tfconf):
  init.run()
  print(a.eval())
  add.eval()

2.0


In [2]:
Image(url="img/service.jpg",height=300)

### 👊 12.2.4 리소스 컨테이너를 사용해 여러 세션에서 상태 공유하기
로컬 세션을 사용할 때는 변수를 세션에서 관리한다.  
그러나 분산 세션을 사용할 때는 클러스터의 리소스 컨테이너에 의해 관리된다.
- 실제로 변수의 저장은 할당된 머신에 저장돼있고 관리를 리소스 컨테이너에서 하게된다.

In [8]:
with tf.Session("grpc://localhost:2222",config=tfconf):
  print(a.eval()) # a.type=="Variable"

4.0


### 👊 12.2.6 큐를 통한 비동기 통신  
큐는 비동기 통신의 좋은 방법이다. 텐서 플로우는 다양한 큐를 제공하는데 FIFO,RandomShuffle,PaddingFifo 등을 제공한다.  
이 큐는 enqueue 와 dequeue를 통하여 데이터를 관리한다. enqueue시 큐가 차거나 dequeue 시에 큐가 비어있으면 기다린다.  
close 함수는 종료 시그널을 보내 데이터 추출의 끝을 알린다.

## 👊 12.3 클러스터에서 신경망 병렬화하기
1. 장치마다 하나의 신경망 배치  
\-- 속도 증가가 선형적, 하이퍼파라미터 튜닝, 대규모 동시 예측  

2.  그래프 내 복제, 그래프 간 복제  
  - 그래프 내 복제  
  하나의 그래프가 입력, 예측, 출력을 담당하고 서버에 분배한다.
  - 그래프 간 복제  
  신경망을 독립된 그래프를 만들고 큐를 이용하여 입력과 출력을 관리한다.
3. 모델 병렬화  
  \-- 완전 연결 신경망의 경우 한번에 하나의 장치만이 계산을 하므로 얻는 이득은 적다.   
  그러나 합성곱 신경망,심층 순환 신경망 등은 실행, 학습속도를 높일 수 있다.  
  
4. 데이터 병렬화 - 각 장치에 모델을 복제하고 다른 미니배치를 사용해 훈련한다. 그래디언트를 휘합하여 모델을 학습시킨다.  
  - 동기 업데이트  
  \-- 그래디언트 수집기가 모든 그래디언트를 수집하고 평균을 계산하여 반영한다.  
      가장 느린 장치의 속도에 맞춰야 하고 동시에 ps에 접속할 것이므로 대역폭을 포화시킬 수 있다. (느린 장치를 무시하여 속도를 높인다. 여분의 복제 모델이라고한다.)
  - 비동기 업데이트  
  \-- 각기의 장치가 계산이 끝날 때마다 모델 파라미터를 업데이트한다.  
  그래디언트가 심하게 흔들린다.

#### 데이터 병렬화 구현
그래프 내 복제 vs 그래프 간 복제  
동기 업데이트 vs 비동기 업데이트  
이 두가지 선택지를 조합하여 총 4가지 조합을 만들 수 있다.


##### 대역폭 포화
데이터의 병렬화는 ps에서 파라미터를 전송해야한다. 이는 대역폭을 포화시키고 네트워크를 느리게 만든다.  
해결 방법으로는  
1. gpu를 몇대의 머신에 모은다.  
2. 여러대의 ps에 파라미터를 분산키니다.  
3. 실수 정밀도를 float32 에서 float16으로 감소시킨다.    
\* 훈련이 끝나고 예측 게산속도를 높이고 크기를 줄이기위해 파라미터 float8로 낮춘다  